# End-to-End Workflow for `DESS`

## Data Prereprocessing

We recieve a file from Dropbox that we convert to parquet and merge to our internal existing files (`completed.parquet`, `uncompleted.parquet` and `reprocessed.parquet`). 
- Convert `.dta` file to parquet
- Based on fullid column, check if it's already been processed (check in `reprocessed.parquet` and `completed.parquet`).
    - If not, add it to `uncompleted.parquet`.
- Add relevant columns to `uncompleted.parquet` (see `prune_dataframe` function)

## Processing Data
Current thinking is that the three high-level functions should be modularized since they may be used in any order.

### Scraping
- Run the processing script on `uncompleted.parquet`.
    - This can be executed either outside the notebook (eg `./main.py`) or inside the notebook (eg: `!caffeinate -dui ./main.py`)
    - You would probably want some way to `get_stats` about the processing (look at `statusUpdate` from `status.ipynb`)
    - Relevant function(s) for this: `process_and_cache(df)`
- Filter out relevant rows and add to `reprocessed.parquet` (based on null rawText).

### Department Extraction
- Call `populate_faculty` on file 
    - By default, the file will be `uncompleted.parquet`.
    - It could also be on `completed.parquet`— when we make changes to our department extraction logic (and want to re-run it on the existing data).

### Data merging
- Want to merge the scraped + faculty-filled information to other internal files
    - Add rows to `completed.parquet` and `reprocessed.parquet`
    - Remove rows from `uncompleted.parquet`

## Post-Processing
- Get a status update about the dataset
    - Want to the completion rates & conversion rates (look at `final_merge.ipynb`?)
- Prepare the completed file for upload
    - convert `completed.parquet` to `.dta` (for uploading to Dropbox)
    - Need to think about how to handle file synchronization and data redundancy for these internal files— easiest solution is push these parquet files to dropbox.

## Other ToDos
- Update the department extraction logic:
    - Re-order the primary patterns (move "the" pattern to top)
    - Add logic to handle isProfessor2 variable
    - Figure out what edge cases to consider